In [1]:
%cd upscalar(ESRGAN)

[WinError 2] The system cannot find the file specified: 'upscalar(ESRGAN)'
c:\Users\alapa\imageupscaler\upscalar(ESRGAN)


C:\Users\alapa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [2]:
import os

# Directory paths
model_dir = 'weights'
input_dir = 'input'
results_dir = 'results'

# Ensure main directories exist
os.makedirs(model_dir, exist_ok=True)
os.makedirs(input_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

# Create a subdirectory in `results` for each model
for model_file in os.listdir(model_dir):
    if model_file.endswith('.pth'):
        model_results_dir = os.path.join(results_dir, model_file)
        os.makedirs(model_results_dir, exist_ok=True)

print("Folder structure set up successfully.")


Folder structure set up successfully.


In [7]:
import tkinter
from tkinter import filedialog
from PIL import Image
import os

# Set the path to your input folder
input_folder = "input"  # Name of the folder where the image will be saved

# Create a Tkinter window
root = tkinter.Tk()


print("Upload image")

# Prompt the user to select an image file
file_path = filedialog.askopenfilename(
    filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.gif;*.bmp")]
)

if not file_path:
    print("No file selected. Exiting.")
    exit()

root.withdraw()  # Hide the main window

# Load the selected image
img = Image.open(file_path)

# Ensure the 'input' folder exists, create it if it doesn't
os.makedirs(input_folder, exist_ok=True)

# Save the image in the input folder with the same name
image_name = os.path.basename(file_path)  # Extract the file name from the selected file path
saved_image_path = os.path.join(input_folder, image_name)  # Path where the image will be saved

# Save the image to the 'input' folder
img.save(saved_image_path)
print(f"Image uploaded and saved to: {saved_image_path}")


Upload image
Image uploaded and saved to: input\264286_00007889.jpg


In [ ]:
import os
import torch
from os.path import join, basename
from PIL import Image
from py_real_esrgan.model import RealESRGAN


# Directory where models are stored
model_dir = "weights"

# Device selection (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load Real-ESRGAN models for each model file in the models directory
for model_file in os.listdir(model_dir):
    print(model_file)
    if model_file.endswith('.pth'):
        model_path = os.path.join(model_dir, model_file)
        result_path = os.path.join(results_dir,model_file)
        # Infer scale from the model filename (2x, 4x, 8x)
        if 'x2' in model_file:
            model_scale = 2
        elif 'x4' in model_file:
            model_scale = 4
        elif 'x8' in model_file:
            model_scale = 8
        else:
            exit()  # Default scale if not specified in the filename

        model = RealESRGAN(device, scale=model_scale)
        model.load_weights( model_path , download=True )
        
        path_to_image = saved_image_path
        image = Image.open(path_to_image).convert('RGB')

        sr_image = model.predict(image)
        output_path = join(result_path , f"output__{basename(file_path)}")
        sr_image.save(output_path)

RealESRGAN_x2.pth
RealESRGAN_x2plus.pth
RealESRGAN_x4.pth
RealESRGAN_x4plus.pth
RealESRGAN_x8.pth
RealESRNet_x4plus.pth


# live video processing

In [ ]:
%cd upscalar(ESRGAN)

In [ ]:
import os
import torch
from os.path import join, basename
from PIL import Image
from py_real_esrgan.model import RealESRGAN

# Directory where models are stored
model_dir = "weights"

# Device selection (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_scale = 4 # 2 or 4 or 8
model_path = 'weights/RealESRNet_x4plus.pth'

model = RealESRGAN(device, scale=model_scale)
model.load_weights( model_path , download=True )

In [ ]:
import cv2
import os
from PIL import Image
import numpy as np
import torch  # Assuming the model is a PyTorch model

# Step 1: Capture live video and save frames to `original_temp_frames`
def capture_live_feed(original_frames_folder, fps=30):
    # Set up video capture for live feed (0 for the default camera)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return
    
    frame_count = 0
    print("Capturing live feed... Press 'q' to quit.")
    
    # Capture loop
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Display the live feed
        cv2.imshow('Live Feed', frame)
        
        # Save each frame temporarily
        frame_count += 1
        frame_path = os.path.join(original_frames_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        
        # Quit with 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Wait to match the target FPS
        cv2.waitKey(int(1000 / fps))
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Captured live feed frames saved to {original_frames_folder}")

# Step 2: Enhance saved frames and create reconstructed video
def enhance_and_reconstruct(original_frames_folder, enhanced_frames_folder, reconstructed_video_path, model, fps=30):
    # Set up video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    
    # Loop through saved frames
    frame_count = 1
    video_writer = None
    
    print("Starting enhancement and reconstruction...")
    while True:
        # Path of the current frame to be enhanced
        frame_path = os.path.join(original_frames_folder, f"frame_{frame_count}.jpg")
        
        # Check if the frame exists
        if not os.path.exists(frame_path):
            break
        
        # Load the frame, enhance it, and save to enhanced folder
        frame_pil = Image.open(frame_path).convert('RGB')
        sr_image = model.predict(frame_pil)  # Enhance with the model
        enhanced_frame = np.array(sr_image)
        
        # Save enhanced frame
        enhanced_frame_path = os.path.join(enhanced_frames_folder, f"frame_{frame_count}.jpg")
        sr_image.save(enhanced_frame_path)
        
        # Initialize video writer if not already set up
        if video_writer is None:
            height, width = enhanced_frame.shape[:2]
            video_writer = cv2.VideoWriter(reconstructed_video_path, fourcc, fps, (width, height))
        
        # Write enhanced frame to output video
        video_writer.write(cv2.cvtColor(enhanced_frame, cv2.COLOR_RGB2BGR))
        
        frame_count += 1

    # Release resources when done
    video_writer.release()
    print(f"Enhanced frames and reconstructed video saved to {enhanced_frames_folder} and {reconstructed_video_path}")

# Set up directories
original_frames_folder = "temp"      # Temporary folder for original frames
enhanced_frames_folder = "temp_enhanced"      # Temporary folder for enhanced frames
reconstructed_video_path = "output/reconstructed_video.mp4"  # Path to save the output video

# Create folders if they don't exist
os.makedirs(original_frames_folder, exist_ok=True)
os.makedirs(enhanced_frames_folder, exist_ok=True)

# Run the capture and enhancement processes in sequence
capture_live_feed(original_frames_folder)
enhance_and_reconstruct(original_frames_folder, enhanced_frames_folder, reconstructed_video_path, model)


In [ ]:
enhance_and_reconstruct(original_frames_folder, enhanced_frames_folder, reconstructed_video_path, model)

# below are test files

In [ ]:
## V1

import cv2
import os

def process_live_feed(output_frames_folder, reconstructed_video_path, fps=30):
    # Set up directories
    os.makedirs(output_frames_folder, exist_ok=True)
    frame_count = 0
    
    # Set up video capture for live feed (0 for the default camera)
    cap = cv2.VideoCapture(0)  
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return
    
    # Get the width and height of the frames
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Set up video writer for real-time reconstructed video
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(reconstructed_video_path, fourcc, fps, (width, height))
    
    print("Processing live feed... Press 'q' to quit.")
    
    # Real-time frame processing loop
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Save each frame temporarily
        frame_count += 1
        frame_path = os.path.join(output_frames_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        
        # Write each frame to the output video
        video_writer.write(frame)
        
        # Remove the frame file immediately after writing to video
        os.remove(frame_path)
        
        # Display the live feed
        cv2.imshow('Live Feed', frame)
        
        # Quit with 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()
    print(f"Reconstructed video saved to {reconstructed_video_path}")

# Example usage
output_frames_folder = "temp"  # Temporary folder for frames
reconstructed_video_path = "output/reconstructed_video.mp4"  # Save path for the output videoq
process_live_feed(output_frames_folder, reconstructed_video_path)


In [ ]:
## V2


import cv2
import os
from PIL import Image
import numpy as np
import torch  # Assuming the model is a PyTorch model

def process_live_feed_with_enhancement(output_frames_folder, reconstructed_video_path, model, fps=30):
    # Set up directories
    os.makedirs(output_frames_folder, exist_ok=True)
    frame_count = 0
    
    # Set up video capture for live feed (0 for the default camera)
    cap = cv2.VideoCapture(0)  
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return
    
    # Get the width and height of the frames
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    # Set up video writer for the reconstructed video
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    video_writer = cv2.VideoWriter(reconstructed_video_path, fourcc, fps, (width, height))
    
    print("Processing live feed... Press 'q' to quit.")
    
    # Real-time frame processing loop
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Display the live feed (original frame)
        cv2.imshow('Live Feed', frame)
        
        # Convert frame to a PIL image for model processing
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        # Enhance frame using the provided model
        sr_image = model.predict(frame_pil)  # Model enhancement

        # Convert enhanced image back to OpenCV format
        enhanced_frame = cv2.cvtColor(np.array(sr_image), cv2.COLOR_RGB2BGR)
        
        # Save each enhanced frame temporarily
        frame_count += 1
        frame_path = os.path.join(output_frames_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, enhanced_frame)
        
        # Write each enhanced frame to the output video
        video_writer.write(enhanced_frame)
        
        # Remove the enhanced frame file immediately after writing to video
        #os.remove(frame_path)
        
        # Quit with 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release resources
    cap.release()
    video_writer.release()
    cv2.destroyAllWindows()
    print(f"Reconstructed video saved to {reconstructed_video_path}")

# Example usage
output_frames_folder = "temp"  # Temporary folder for frames
reconstructed_video_path = "output/reconstructed_video.mp4"  # Save path for the output video
process_live_feed_with_enhancement(output_frames_folder, reconstructed_video_path, model)


In [ ]:
## V3

import cv2
import os
from PIL import Image
import numpy as np
import threading
import time
import torch  # Assuming the model is a PyTorch model

# Function for capturing live feed and saving frames to `original_temp_frames`
def capture_live_feed(original_frames_folder, fps=30):
    # Set up video capture for live feed (0 for the default camera)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return
    
    frame_count = 0
    print("Capturing live feed... Press 'q' to quit.")
    
    # Capture loop
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Display the live feed
        cv2.imshow('Live Feed', frame)
        
        # Save each frame temporarily
        frame_count += 1
        frame_path = os.path.join(original_frames_folder, f"frame_{frame_count}.jpg")
        cv2.imwrite(frame_path, frame)
        
        # Quit with 'q' key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # Wait to match the target FPS
        time.sleep(1 / fps)
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"Captured live feed frames saved to {original_frames_folder}")

# Function for loading frames, enhancing, and saving to `enhanced_temp_frames` and video
def enhance_and_reconstruct(original_frames_folder, enhanced_frames_folder, reconstructed_video_path, model, fps=30):
    # Set up video writer
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    
    # Loop to process frames in real time
    frame_count = 1
    video_writer = None
    
    print("Starting enhancement and reconstruction...")
    while True:
        # Path of the current frame to be enhanced
        frame_path = os.path.join(original_frames_folder, f"frame_{frame_count}.jpg")
        
        # Wait for the frame to be available in the folder
        if not os.path.exists(frame_path):
            time.sleep(0.05)
            continue
        
        # Load the frame, enhance it, and save to enhanced folder
        frame_pil = Image.open(frame_path).convert('RGB')
        sr_image = model.predict(frame_pil)  # Enhance with the model
        enhanced_frame = np.array(sr_image)
        
        # Save enhanced frame
        enhanced_frame_path = os.path.join(enhanced_frames_folder, f"frame_{frame_count}.jpg")
        sr_image.save(enhanced_frame_path)
        
        # Initialize video writer if not already set up
        if video_writer is None:
            height, width = enhanced_frame.shape[:2]
            video_writer = cv2.VideoWriter(reconstructed_video_path, fourcc, fps, (width, height))
        
        # Write enhanced frame to output video
        video_writer.write(cv2.cvtColor(enhanced_frame, cv2.COLOR_RGB2BGR))
        
        frame_count += 1

    # Release resources when done
    video_writer.release()
    print(f"Enhanced frames and reconstructed video saved to {enhanced_frames_folder} and {reconstructed_video_path}")

# Set up directories
original_frames_folder = "temp"      # Temporary folder for original frames
enhanced_frames_folder = "temp_enhanced"      # Temporary folder for enhanced frames
reconstructed_video_path = "output/reconstructed_video.mp4"  # Path to save the output video

# Create folders if they don't exist
os.makedirs(original_frames_folder, exist_ok=True)
os.makedirs(enhanced_frames_folder, exist_ok=True)

# Load your pre-trained PyTorch enhancement model here
# Example: Your PyTorch image enhancement model

# Run both capture and enhancement processes in separate threads
capture_thread = threading.Thread(target=capture_live_feed, args=(original_frames_folder,))
enhance_thread = threading.Thread(target=enhance_and_reconstruct, args=(original_frames_folder, enhanced_frames_folder, reconstructed_video_path, model))

# Start threads
capture_thread.start()
enhance_thread.start()

# Wait for threads to complete
capture_thread.join()
enhance_thread.join()
